## 第一部分：了解 nn.Module的基本操作

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

In [ ]:
model = models.resnet18()

### 打印出 model底下所有 parameters 的 name 以及對應的 shape 

In [14]:
for name, param in model.named_parameters():
  print(name, param.size())

sequential.0.weight torch.Size([10, 12])
sequential.0.bias torch.Size([10])
sequential.1.weight torch.Size([10])
sequential.1.bias torch.Size([10])
repeat_linear.0.0.weight torch.Size([10, 10])
repeat_linear.0.0.bias torch.Size([10])
repeat_linear.0.1.weight torch.Size([10])
repeat_linear.0.1.bias torch.Size([10])
repeat_linear.1.0.weight torch.Size([10, 10])
repeat_linear.1.0.bias torch.Size([10])
repeat_linear.1.1.weight torch.Size([10])
repeat_linear.1.1.bias torch.Size([10])
repeat_linear.2.0.weight torch.Size([10, 10])
repeat_linear.2.0.bias torch.Size([10])
repeat_linear.2.1.weight torch.Size([10])
repeat_linear.2.1.bias torch.Size([10])
repeat_linear.3.0.weight torch.Size([10, 10])
repeat_linear.3.0.bias torch.Size([10])
repeat_linear.3.1.weight torch.Size([10])
repeat_linear.3.1.bias torch.Size([10])
repeat_linear.4.0.weight torch.Size([10, 10])
repeat_linear.4.0.bias torch.Size([10])
repeat_linear.4.1.weight torch.Size([10])
repeat_linear.4.1.bias torch.Size([10])
repeat_linea

### 為了使 forward propagation 加速 並降低 memory 使用量，請將所有 parameters 的requires_grad 關閉，關閉之後執行  forward propagation

In [ ]:
input_ = torch.randn(1, 3, 128, 128)

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
output = model(input_)
print(output.shape)

torch.Size([1, 1000])


## 第二部分：依照指令，以較簡潔的方式搭建出模型

* input_shape = torch.Size([10, 12])
* 先經過一層 nn.Linear(12, 10)
* 經過10層 nn.Linear(10, 10)
* 最後經過 nn.Linear(10, 3) 輸出
* 每一個 nn.Linear過完後要先經過 nn.BatchNorm1d 才能到下一層，輸出層不用


In [ ]:
input_ = torch.randn(10,12)
## 示範
Linear = nn.Linear(12,10)
BN = nn.BatchNorm1d(10)

In [15]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.sequential = nn.Sequential(
            nn.Linear(12, 10), 
            nn.BatchNorm1d(10))
        self.repeat_linear = nn.ModuleList([
                                            nn.Sequential(nn.Linear(10, 10), 
                                                          nn.BatchNorm1d(10))
                                             for _ in range(10)
                                                          ])
        self.output = nn.Linear(10, 3)

    def forward(self, x):
        x = self.sequential(x)
        for module in self.repeat_linear:
          x = module(x)
        x = self.output(x)
        return x

In [16]:
model = Model()

In [17]:
model

Model(
  (sequential): Sequential(
    (0): Linear(in_features=12, out_features=10, bias=True)
    (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (repeat_linear): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (4): Sequential(
      (0): Linear(in_features=10, out_fe

In [18]:
input_ = torch.randn(10,12)
outupt = model(input_)

In [19]:
outupt

tensor([[ 0.3004,  1.0703,  0.3561],
        [ 0.3464,  0.6323, -0.0438],
        [-0.5464,  0.4052,  0.0112],
        [ 0.9146, -1.1665, -0.0643],
        [-1.2983,  0.6350, -0.0041],
        [-0.1500,  0.7993, -0.5536],
        [ 0.4797,  0.2540,  0.4558],
        [-0.3438,  0.7136,  0.5538],
        [ 0.8955, -0.3057, -0.5703],
        [ 0.7662, -0.3459,  0.8192]], grad_fn=<AddmmBackward>)